In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
train_lre = pd.read_csv('/home/satishk/ivectors_csv_new/train_feat_BNF_h5_14oct_Bharat_160k.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
train_lre.head()

,0,1,2,3,4,5,6,7,8,9,...,491,492,493,494,495,496,497,498,499,langid
0,1.120473,-1.207826,-1.102705,0.731326,-1.690868,1.067887,-0.014018,0.159593,0.247715,0.264993,...,-0.094636,-0.323671,0.844373,-0.366620,1.634246,0.441962,0.681505,-0.632136,-0.712648,spa-eur
1,0.855223,0.237345,-0.509226,0.297071,-1.554144,0.310795,0.177555,-1.708162,1.859202,2.637373,...,-1.858368,0.495619,-1.029202,1.921910,-0.779570,2.633960,0.226205,0.493565,-0.376497,spa-eur
2,2.178442,-0.152923,-0.277384,1.359762,-1.063728,0.251058,-0.175947,0.767762,0.104705,1.704032,...,1.819529,-1.252623,0.706020,-0.369342,-0.630677,1.568804,1.663335,0.067233,0.539783,spa-eur
3,0.020323,0.186472,-1.045840,1.076967,-0.881765,0.140140,0.056690,-1.387089,0.074638,1.077490,...,1.402422,1.356642,-0.030127,-1.133595,-1.550995,-1.052241,1.623444,-0.244312,0.853134,spa-eur
4,-1.672608,1.260625,-0.865485,1.902568,1.251961,1.508542,-0.085114,3.884414,-1.814981,0.504477,...,0.000128,0.103537,-0.044487,1.756438,2.346587,-0.574275,-0.874379,0.378014,-1.190706,spa-eur


In [5]:
train_lre.groupby(['langid']).size()

langid
ara-acm     4380
ara-apc     5500
ara-ary     4467
ara-arz    23779
eng-gbr     1104
eng-usg    14529
por-brz      962
qsl-pol     4258
qsl-rus    11552
spa-car    23799
spa-eur     2807
spa-lac    24662
zho-cmn    38465
zho-nan     3126
dtype: int64

In [6]:
val_lre = pd.read_csv('/home/satishk/ivectors_csv_new/dev_feat_BNF_h5_3661_uttid_Bharat_160k.csv')


In [7]:
#val_lre = val_lre.iloc[100:300]

In [8]:
val_lre.head()

,segmentid,language_code,data_source,speech_duration,0,1,2,3,4,5,...,491,492,493,494,495,496,497,498,499,uttid
0,lre17_ntrlosgu.sph,ara-acm,mls14,30,1.098584,1.575492,-0.453370,-0.651831,0.285572,-0.726594,...,0.458423,-1.359055,0.640751,0.094361,-0.700234,-0.034466,-0.580875,0.004446,-0.104522,lre17_ntrlosgu
1,lre17_moxnwuqe.sph,ara-acm,mls14,10,1.436526,1.444811,-0.589956,-0.255953,0.850315,-0.745601,...,1.388978,-1.977061,2.114798,0.012926,-1.669853,0.446671,-2.128618,-0.970636,0.766287,lre17_moxnwuqe
2,lre17_meesvkxz.sph,ara-acm,mls14,3,0.550556,1.410690,-0.617443,0.943157,0.864899,-0.206588,...,-0.336275,-2.661029,0.086109,0.951649,-0.078517,0.722322,-1.811978,-1.288542,-0.250591,lre17_meesvkxz
3,lre17_rqmsmzui.sph,ara-acm,mls14,30,0.785263,1.225041,-0.615307,-0.601939,0.067760,-0.303262,...,0.712126,-0.269891,-0.210374,-0.675105,-0.101579,0.432103,-0.307661,-0.430126,-1.499074,lre17_rqmsmzui
4,lre17_qgszpuyw.sph,ara-acm,mls14,10,1.561980,1.057819,-0.846207,-0.226456,-0.017701,-0.419544,...,0.590097,-1.033919,-0.465676,-0.503004,-1.776944,0.901027,0.513566,0.419297,-0.321541,lre17_qgszpuyw


In [9]:
X_train = train_lre.drop("langid",axis=1)
y_train = train_lre["langid"]
#y_train_uttid = train_lre["uttid"]

In [10]:
X_val = val_lre.drop(["language_code","uttid","segmentid","data_source","speech_duration"],axis=1)
y_val = val_lre["language_code"]
y_val_segmentid = val_lre["segmentid"]

In [11]:
X_train = X_train.append(X_val, ignore_index=True)
y_train = y_train.append(y_val, ignore_index=True)

In [12]:
train_lre.groupby(['langid']).size()

langid
ara-acm     4380
ara-apc     5500
ara-ary     4467
ara-arz    23779
eng-gbr     1104
eng-usg    14529
por-brz      962
qsl-pol     4258
qsl-rus    11552
spa-car    23799
spa-eur     2807
spa-lac    24662
zho-cmn    38465
zho-nan     3126
dtype: int64

In [13]:
y_train = y_train.replace(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr','eng-usg', 'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car','spa-eur', 'spa-lac', 'zho-cmn', 'zho-nan'], 
                     [0, 0, 0, 0, 1,0, 0, 0, 0, 0,0, 0, 0, 0]) 

In [14]:
y_val = y_val.replace(['ara-acm', 'ara-apc', 'ara-ary', 'ara-arz', 'eng-gbr','eng-usg', 'por-brz', 'qsl-pol', 'qsl-rus', 'spa-car','spa-eur', 'spa-lac', 'zho-cmn', 'zho-nan'], 
                     [0, 0, 0, 0, 1,0, 0, 0, 0, 0,0, 0, 0, 0]) 

In [15]:
le = preprocessing.LabelEncoder()
le.fit(y_train)

LabelEncoder()

In [16]:
le.classes_

array([0, 1])

In [17]:
y_train=le.transform(y_train)

In [18]:
le = preprocessing.LabelEncoder()
le.fit(y_val)

LabelEncoder()

In [19]:
le.classes_

array([0, 1])

In [20]:
X_train=X_train.values
X_val=X_val.values

In [21]:
nb_classes = 2

In [22]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [23]:
Y_train = indices_to_one_hot(y_train, 2)
#Y_test = indices_to_one_hot(y_test, 14)
Y_val = indices_to_one_hot(y_val, 2)

In [24]:
Y_val

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [25]:
Y_train

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [26]:
Y_val.shape

(3661, 2)

In [27]:
Y_train.shape

(167051, 2)

In [28]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
X_val = X_val.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
#print(X_test.shape[0], 'test samples')
print(X_val.shape[0], 'val samples')


X_train shape: (167051, 500)
167051 train samples
3661 val samples


In [29]:
X_train.shape

(167051, 500)

In [30]:
y_train.shape

(167051,)

In [31]:
X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [32]:
X_val.shape

(3661, 500)

In [33]:
#mnist = input_data.read_data_sets('/home/satishk/depy_04_AUG/MNIST_data', one_hot=True)
mb_size = 256
Z_dim = 100
X_dim = 500 #mnist.train.images.shape[1]
y_dim = 2 #mnist.train.labels.shape[1]
h_dim = 128
cnt = 0
lr = 1e-3


In [34]:
X_dim, y_dim

(500, 2)

In [35]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [36]:
""" ==================== GENERATOR ======================== """

Wzh = xavier_init(size=[Z_dim + y_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)


def G(z, c):
    inputs = torch.cat([z, c], 1)
    h = nn.relu(inputs @ Wzh + bzh.repeat(inputs.size(0), 1))
    X = nn.sigmoid(h @ Whx + bhx.repeat(h.size(0), 1))
    return X



In [37]:
""" ==================== DISCRIMINATOR ======================== """

Wxh = xavier_init(size=[X_dim + y_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

Why = xavier_init(size=[h_dim, 1])
bhy = Variable(torch.zeros(1), requires_grad=True)


def D(X, c):
    inputs = torch.cat([X, c], 1)
    h = nn.relu(inputs @ Wxh + bxh.repeat(inputs.size(0), 1))
    y = nn.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    return y


G_params = [Wzh, bzh, Whx, bhx]
D_params = [Wxh, bxh, Why, bhy]
params = G_params + D_params

In [38]:

""" ===================== TRAINING ======================== """


def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())


G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

#ones_label = Variable(torch.ones(mb_size))
#zeros_label = Variable(torch.zeros(mb_size))

ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))


In [39]:
len(X_train)

167051

In [40]:
batch_size = mb_size
# Start training
for epoch in range(25):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        c = Y_train[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(z, c)
        D_real = D(X, c)
        D_fake = D(G_sample, c)

        D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        D_loss = D_loss_real + D_loss_fake

        D_loss.backward()
        D_solver.step()

        # Housekeeping - reset gradient
        reset_grad()

        # Generator forward-loss-backward-update
        z = Variable(torch.randn(mb_size, Z_dim))
        G_sample = G(z, c)
        D_fake = D(G_sample, c)

        G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()

        # Housekeeping - reset gradient
        reset_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.numpy(), G_loss.data.numpy()))



        it+= batch_size
           

epoch: 0


/home/satishk/miniconda2/envs/lre17/lib/python3.6/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Iter-0; D_loss: [ 2.00384378]; G_loss: [ 1.15448952]
Iter-256; D_loss: [ 1.37788844]; G_loss: [ 1.97938669]
Iter-512; D_loss: [ 1.03245425]; G_loss: [ 2.82330704]
Iter-768; D_loss: [ 0.9487502]; G_loss: [ 3.5344131]
Iter-1024; D_loss: [ 0.81388468]; G_loss: [ 4.16800833]
Iter-1280; D_loss: [ 0.74363333]; G_loss: [ 4.58556652]
Iter-1536; D_loss: [ 0.70090145]; G_loss: [ 4.98492908]
Iter-1792; D_loss: [ 0.64033562]; G_loss: [ 5.15185499]
Iter-2048; D_loss: [ 0.62255335]; G_loss: [ 5.35943985]
Iter-2304; D_loss: [ 0.60056466]; G_loss: [ 5.40311193]
Iter-2560; D_loss: [ 0.63475519]; G_loss: [ 5.44257498]
Iter-2816; D_loss: [ 0.52838063]; G_loss: [ 5.4110713]
Iter-3072; D_loss: [ 0.51182514]; G_loss: [ 5.32859516]
Iter-3328; D_loss: [ 0.43364155]; G_loss: [ 5.24825144]
Iter-3584; D_loss: [ 0.39797342]; G_loss: [ 5.14242268]
Iter-3840; D_loss: [ 0.41633055]; G_loss: [ 4.91989183]
Iter-4096; D_loss: [ 0.37486753]; G_loss: [ 4.79438114]
Iter-4352; D_loss: [ 0.35196856]; G_loss: [ 4.7046299]
It

Iter-37376; D_loss: [ 0.3000167]; G_loss: [ 1.78257251]
Iter-37632; D_loss: [ 0.28020671]; G_loss: [ 1.78118825]
Iter-37888; D_loss: [ 0.31186342]; G_loss: [ 1.80135918]
Iter-38144; D_loss: [ 0.28169841]; G_loss: [ 1.79964149]
Iter-38400; D_loss: [ 0.26540357]; G_loss: [ 1.79975033]
Iter-38656; D_loss: [ 0.28360662]; G_loss: [ 1.79244602]
Iter-38912; D_loss: [ 0.30826896]; G_loss: [ 1.81638241]
Iter-39168; D_loss: [ 0.29202119]; G_loss: [ 1.82777369]
Iter-39424; D_loss: [ 0.27982977]; G_loss: [ 1.78588331]
Iter-39680; D_loss: [ 0.26701707]; G_loss: [ 1.83526099]
Iter-39936; D_loss: [ 0.30574146]; G_loss: [ 1.83640623]
Iter-40192; D_loss: [ 0.26191062]; G_loss: [ 1.831581]
Iter-40448; D_loss: [ 0.26174802]; G_loss: [ 1.80574477]
Iter-40704; D_loss: [ 0.29287103]; G_loss: [ 1.8459816]
Iter-40960; D_loss: [ 0.2548092]; G_loss: [ 1.82941711]
Iter-41216; D_loss: [ 0.26221591]; G_loss: [ 1.84579611]
Iter-41472; D_loss: [ 0.25222632]; G_loss: [ 1.87182343]
Iter-41728; D_loss: [ 0.25106585]; G

Iter-75008; D_loss: [ 0.08802019]; G_loss: [ 2.7253654]
Iter-75264; D_loss: [ 0.08860892]; G_loss: [ 2.73540378]
Iter-75520; D_loss: [ 0.08830352]; G_loss: [ 2.71866465]
Iter-75776; D_loss: [ 0.09905731]; G_loss: [ 2.69932866]
Iter-76032; D_loss: [ 0.08526964]; G_loss: [ 2.7007134]
Iter-76288; D_loss: [ 0.09011577]; G_loss: [ 2.69950986]
Iter-76544; D_loss: [ 0.0886793]; G_loss: [ 2.68561888]
Iter-76800; D_loss: [ 0.09080831]; G_loss: [ 2.68171477]
Iter-77056; D_loss: [ 0.08724467]; G_loss: [ 2.69249773]
Iter-77312; D_loss: [ 0.09099841]; G_loss: [ 2.68348479]
Iter-77568; D_loss: [ 0.08828104]; G_loss: [ 2.695328]
Iter-77824; D_loss: [ 0.09253222]; G_loss: [ 2.71516252]
Iter-78080; D_loss: [ 0.09085741]; G_loss: [ 2.73517895]
Iter-78336; D_loss: [ 0.08468933]; G_loss: [ 2.7619276]
Iter-78592; D_loss: [ 0.07840107]; G_loss: [ 2.78788447]
Iter-78848; D_loss: [ 0.08680269]; G_loss: [ 2.82360816]
Iter-79104; D_loss: [ 0.08481901]; G_loss: [ 2.85476136]
Iter-79360; D_loss: [ 0.07652944]; G_

...

Iter-8192; D_loss: [  1.73539846e-07]; G_loss: [ 15.7300396]
Iter-8448; D_loss: [  1.88415655e-07]; G_loss: [ 15.67853355]
Iter-8704; D_loss: [  1.85794150e-07]; G_loss: [ 15.70179081]
Iter-8960; D_loss: [  1.88226565e-07]; G_loss: [ 15.68074512]
Iter-9216; D_loss: [  1.88560307e-07]; G_loss: [ 15.68010998]
Iter-9472; D_loss: [  2.09067395e-07]; G_loss: [ 15.65335846]
Iter-9728; D_loss: [  1.76346518e-07]; G_loss: [ 15.63717747]
Iter-9984; D_loss: [  1.77916945e-07]; G_loss: [ 15.61108017]
Iter-10240; D_loss: [  1.96255755e-07]; G_loss: [ 15.59481716]
Iter-10496; D_loss: [  1.84566659e-07]; G_loss: [ 15.57312584]
Iter-10752; D_loss: [  2.06290920e-07]; G_loss: [ 15.63096046]
Iter-11008; D_loss: [  1.93141389e-07]; G_loss: [ 15.60194016]
Iter-11264; D_loss: [  1.94827521e-07]; G_loss: [ 15.56150723]
Iter-11520; D_loss: [  1.98982249e-07]; G_loss: [ 15.64501858]
Iter-11776; D_loss: [  2.08633494e-07]; G_loss: [ 15.62604713]
Iter-12032; D_loss: [  2.03562394e-07]; G_loss: [ 15.65331173]
I

Iter-42496; D_loss: [  9.53378617e-07]; G_loss: [ 15.46297264]
Iter-42752; D_loss: [  1.16257991e-06]; G_loss: [ 15.34422016]
Iter-43008; D_loss: [  1.09491589e-06]; G_loss: [ 15.24776936]
Iter-43264; D_loss: [  1.01369358e-06]; G_loss: [ 15.21505451]
Iter-43520; D_loss: [  1.27117369e-06]; G_loss: [ 15.4128046]
Iter-43776; D_loss: [  1.22296899e-06]; G_loss: [ 15.22626877]
Iter-44032; D_loss: [  1.39807673e-06]; G_loss: [ 15.1337719]
Iter-44288; D_loss: [  1.01334228e-06]; G_loss: [ 15.27006626]
Iter-44544; D_loss: [  1.29683860e-06]; G_loss: [ 15.19459438]
Iter-44800; D_loss: [  1.72766829e-06]; G_loss: [ 15.21387577]
Iter-45056; D_loss: [  9.96614517e-07]; G_loss: [ 15.22776318]
Iter-45312; D_loss: [  1.12178725e-06]; G_loss: [ 15.30937386]
Iter-45568; D_loss: [  7.67933898e-07]; G_loss: [ 15.3690958]
Iter-45824; D_loss: [  7.10016991e-07]; G_loss: [ 15.4875803]
Iter-46080; D_loss: [  7.62202092e-07]; G_loss: [ 15.28198719]
Iter-46336; D_loss: [  9.61856813e-07]; G_loss: [ 15.468743

Iter-76544; D_loss: [  4.76142901e-07]; G_loss: [ 15.048419]
Iter-76800; D_loss: [  5.51483424e-07]; G_loss: [ 15.10993767]
Iter-77056; D_loss: [  5.16056332e-07]; G_loss: [ 15.10079384]
Iter-77312; D_loss: [  6.31923456e-07]; G_loss: [ 14.92676353]
Iter-77568; D_loss: [  5.39016867e-07]; G_loss: [ 14.97636414]
Iter-77824; D_loss: [  6.20013850e-07]; G_loss: [ 14.8942976]
Iter-78080; D_loss: [  7.11054156e-07]; G_loss: [ 14.95463467]
Iter-78336; D_loss: [  1.04799437e-06]; G_loss: [ 14.9241848]
Iter-78592; D_loss: [  8.49394382e-07]; G_loss: [ 14.87489414]
Iter-78848; D_loss: [  9.73152510e-07]; G_loss: [ 14.85758114]
Iter-79104; D_loss: [  1.32146590e-06]; G_loss: [ 14.86301613]
Iter-79360; D_loss: [  1.03914090e-06]; G_loss: [ 14.95719147]
Iter-79616; D_loss: [  1.21445987e-06]; G_loss: [ 14.69524193]
Iter-79872; D_loss: [  2.00092063e-06]; G_loss: [ 14.79279709]
Iter-80128; D_loss: [  1.94905920e-06]; G_loss: [ 14.85211277]
Iter-80384; D_loss: [  1.65254471e-06]; G_loss: [ 14.654428

Iter-110336; D_loss: [  1.37423626e-06]; G_loss: [ 14.89196491]
Iter-110592; D_loss: [  1.10806661e-06]; G_loss: [ 15.02133083]
Iter-110848; D_loss: [  3.37425990e-06]; G_loss: [ 14.97462368]
Iter-111104; D_loss: [  1.04753008e-06]; G_loss: [ 14.99976635]
Iter-111360; D_loss: [  8.73640261e-07]; G_loss: [ 15.17796135]
Iter-111616; D_loss: [  9.42303643e-07]; G_loss: [ 15.09271717]
Iter-111872; D_loss: [  7.75403350e-07]; G_loss: [ 15.09703732]
Iter-112128; D_loss: [  8.52073811e-07]; G_loss: [ 15.18578911]
Iter-112384; D_loss: [  7.87443071e-07]; G_loss: [ 15.1001091]
Iter-112640; D_loss: [  7.66193125e-07]; G_loss: [ 15.30557919]
Iter-112896; D_loss: [  8.59850900e-07]; G_loss: [ 15.18003941]
Iter-113152; D_loss: [  9.72414455e-07]; G_loss: [ 15.13939857]
Iter-113408; D_loss: [  1.04112792e-06]; G_loss: [ 15.19778156]
Iter-113664; D_loss: [  1.89586910e-06]; G_loss: [ 15.13054276]
Iter-113920; D_loss: [  6.58941701e-07]; G_loss: [ 15.18481636]
Iter-114176; D_loss: [  1.15911519e-06]; 

Iter-143872; D_loss: [  6.06484946e-07]; G_loss: [ 14.82277966]
Iter-144128; D_loss: [  7.23900769e-07]; G_loss: [ 14.93012428]
Iter-144384; D_loss: [  8.31676573e-07]; G_loss: [ 14.80633736]
Iter-144640; D_loss: [  8.63446303e-07]; G_loss: [ 14.697052]
Iter-144896; D_loss: [  9.14845089e-07]; G_loss: [ 14.64987183]
Iter-145152; D_loss: [  1.07703261e-06]; G_loss: [ 14.5915184]
Iter-145408; D_loss: [  1.06485709e-06]; G_loss: [ 14.50437069]
Iter-145664; D_loss: [  1.27843066e-06]; G_loss: [ 14.3260603]
Iter-145920; D_loss: [  1.87225430e-06]; G_loss: [ 14.18371105]
Iter-146176; D_loss: [  1.57160389e-06]; G_loss: [ 14.38381577]
Iter-146432; D_loss: [  2.03309378e-06]; G_loss: [ 13.98258495]
Iter-146688; D_loss: [  2.15462228e-06]; G_loss: [ 13.76097393]
Iter-146944; D_loss: [  2.66755774e-06]; G_loss: [ 13.87388897]
Iter-147200; D_loss: [  4.21561163e-06]; G_loss: [ 13.75617409]
Iter-147456; D_loss: [  2.92385721e-06]; G_loss: [ 13.64723492]
Iter-147712; D_loss: [  2.76834089e-06]; G_l

In [42]:
df_SF=pd.DataFrame()
samples_per_class = 50000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(2):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [43]:
c_df.shape

(50000, 2)

In [44]:
df_SF.shape[0]

100000

In [45]:
c_gen = df_SF.values

In [46]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [47]:
z_gen.data.shape

torch.Size([100000, 100])

In [48]:
c_gen = Variable(torch.from_numpy(c_gen))

In [49]:
samples = G(z_gen, c_gen)

In [50]:
samples.data.shape

torch.Size([100000, 500])

In [51]:
#Taking the generated iVectors we will try to check the acc by MLP

In [52]:
train_X = samples.data.numpy()

In [53]:
type(train_X)

numpy.ndarray

In [54]:
train_y = c_gen.data.numpy()

In [55]:
type(train_y)

numpy.ndarray

In [56]:
train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [57]:
model = Sequential()
model.add(Dense(64, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [58]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                32064     
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 130       
__________

In [59]:
adam=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [60]:
best_weights_filepath = '/home/satishk/Desktop/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [61]:
batch_size = 256
nb_epoch=15

In [62]:
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_val , Y_val),callbacks=[saveBestModel])

Train on 100000 samples, validate on 3661 samples
Epoch 1/15
Epoch 00000: val_acc improved from -inf to 0.51106, saving model to /home/satishk/Desktop/best_weights_2l_MLP_11.hdf5
1s - loss: 0.6959 - acc: 0.5246 - val_loss: 0.8550 - val_acc: 0.5111
Epoch 2/15
Epoch 00001: val_acc did not improve
1s - loss: 0.6840 - acc: 0.5510 - val_loss: 1.3682 - val_acc: 0.4215
Epoch 3/15
Epoch 00002: val_acc did not improve
1s - loss: 0.6813 - acc: 0.5588 - val_loss: 1.7286 - val_acc: 0.4040
Epoch 4/15
Epoch 00003: val_acc did not improve
1s - loss: 0.6805 - acc: 0.5602 - val_loss: 2.2240 - val_acc: 0.3384
Epoch 5/15
Epoch 00004: val_acc did not improve
1s - loss: 0.6800 - acc: 0.5609 - val_loss: 2.5092 - val_acc: 0.3018
Epoch 6/15
Epoch 00005: val_acc did not improve
1s - loss: 0.6799 - acc: 0.5600 - val_loss: 2.6885 - val_acc: 0.3125
Epoch 7/15
Epoch 00006: val_acc did not improve
1s - loss: 0.6790 - acc: 0.5631 - val_loss: 2.7423 - val_acc: 0.3349
Epoch 8/15
Epoch 00007: val_acc did not improve
1s

In [63]:
#Frame label accuracy
score = model.evaluate(X_val, Y_val, verbose=0)
print('Acc:', score[1])

Acc: 0.277519803336
